In [1]:
from matplotlib.transforms import Affine2D
import mpl_toolkits.axisartist.floating_axes as floating_axes
import numpy as np
import mpl_toolkits.axisartist.angle_helper as angle_helper
from matplotlib.projections import PolarAxes
from mpl_toolkits.axisartist.grid_finder import (FixedLocator, MaxNLocator,
                                                 DictFormatter)
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
from sklearn import preprocessing
from highlight_text import htext
from matplotlib import cm
import matplotlib.patheffects as path_effects

In [2]:
df = pd.read_html("https://fbref.com/en/comps/Big5/stats/players/Big-5-European-Leagues-Stats")[0]
pd.set_option('display.max_columns', None)

In [5]:
df = df.droplevel(0, axis=1)

In [6]:
df

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,xG,npxG,xA,xG,xA,xG+xA,npxG,npxG+xA,Matches
0,1,Patrick van Aanholt,nl NED,DF,Crystal Palace,eng Premier League,30-060,1990,1,0,19,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.0,0.25,0.00,0.25,0.25,0.25,Matches
1,2,Issah Abbas,gh GHA,FW,Mainz 05,de Bundesliga,22-032,1998,1,0,4,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches
2,3,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,33-030,1987,7,7,630,1,0,0,0,1,0,0.14,0.00,0.14,0.14,0.14,0.3,0.3,0.0,0.05,0.00,0.05,0.05,0.05,Matches
3,4,Mehdi Abeid,dz ALG,MF,Nantes,fr Ligue 1,28-083,1992,5,3,303,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.3,0.3,0.4,0.08,0.11,0.19,0.08,0.19,Matches
4,5,Laurent Abergel,fr FRA,"MF,FW",Lorient,fr Ligue 1,27-270,1993,8,6,639,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.2,0.02,0.02,0.04,0.02,0.04,Matches
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,2174,Steven Zuber,ch SUI,DF,Eint Frankfurt,de Bundesliga,29-072,1991,5,3,320,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.1,0.1,0.5,0.04,0.15,0.19,0.04,0.19,Matches
2260,2175,Martín Zubimendi,es ESP,MF,Real Sociedad,es La Liga,21-269,1999,5,3,284,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.01,0.01,0.00,0.01,Matches
2261,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls,Ast,G+A,G-PK,G+A-PK,xG,npxG,xA,xG,xA,xG+xA,npxG,npxG+xA,Matches
2262,2176,Martin Ødegaard,no NOR,MF,Real Madrid,es La Liga,21-316,1998,3,2,115,0,0,0,0,0,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.2,0.02,0.13,0.15,0.02,0.15,Matches


In [8]:
df = df[['Player','Pos','Squad','Comp','Age','Min', 'xG', 'xA']]

In [9]:
# df = pd.merge(df, df1[cols_to_use], left_index=True, right_index=True, how='outer')

df['Min'] = df['Min'].apply(pd.to_numeric, errors='coerce')
df['xG'] = df['xG'].apply(pd.to_numeric, errors='coerce')
df['xA'] = df['xA'].apply(pd.to_numeric, errors='coerce')


In [11]:
# Remove non-numerical data
all_columns = list(df.columns)
unwanted_columns = {'Player', 'Pos', 'Squad', 'Comp', 'Age','Min'}
index_columns = [ele for ele in all_columns if ele not in unwanted_columns]

In [13]:
dfP90 = df.loc[:,index_columns].div(df['Min'], axis=0)

In [14]:
# P90 Dataframe
master_df = pd.concat([df[['Player', 'Pos', 'Squad', 'Comp', 'Age', 'Min']], dfP90], axis = 1)
master_df.head(5)

,Player,Pos,Squad,Comp,Age,Min,xG,xG,xG,xG,xA,xA,xA,xA
0,Patrick van Aanholt,DF,Crystal Palace,eng Premier League,30-060,19.0,0.005263,0.013158,0.005263,0.013158,0.000000,0.000000,0.000000,0.000000
1,Issah Abbas,FW,Mainz 05,de Bundesliga,22-032,4.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Yunis Abdelhamid,DF,Reims,fr Ligue 1,33-030,630.0,0.000476,0.000079,0.000476,0.000079,0.000000,0.000000,0.000000,0.000000
3,Mehdi Abeid,MF,Nantes,fr Ligue 1,28-083,303.0,0.000990,0.000264,0.000990,0.000264,0.001320,0.000363,0.001320,0.000363
4,Laurent Abergel,"MF,FW",Lorient,fr Ligue 1,27-270,639.0,0.000156,0.000031,0.000156,0.000031,0.000313,0.000031,0.000313,0.000031


In [ ]:
master_df

In [ ]:
players = master_df['Player'].tolist()

In [ ]:
# Drop columns
df1 = master_df.drop(['Player', 'Pos', 'Squad', 'Comp', 'Age', '90s'], axis=1)

In [ ]:
#Normalize data
DFNom = df1.transform(lambda x: (x - x.min())/ (x.max() - x.min()))

In [ ]:
master_df = pd.concat([master_df[['Player','Pos','Squad','Comp','Age']], DFNom], axis = 1)
master_df.head()

In [ ]:
master_df.dtypes

In [ ]:
high = (master_df['Carries'] > 0.5) & (master_df['Possession Regains'] > 0.5)
high_volume = master_df.loc[high]
high_volume

In [ ]:
name = (master_df.Player == 'Thomas Partey')
partey = master_df.loc[name]
partey

In [ ]:
longlist = ((master_df['Carries'] < 0.5) & (master_df['Possession Regains'] < 0.5))
longlist = master_df.loc[longlist]
longlist

In [ ]:
high_pass = (master_df['Carries per Touch'] > 0.5) & ((master_df['Possession Regains'] > 0) & (master_df['Possession Regains'] < 0.5))
high_passer = master_df.loc[high_pass]
high_passer.head()

In [ ]:
high_carry = (master_df['Possession Regains'] > 0.5) & ((master_df['Possession Regains'] > 0) & (master_df['Carries per Touch'] < 0.5))
high_carrier = master_df.loc[high_carry]
high_carrier

In [ ]:
def setup_axes1(fig, rect):
    """
    Rotated Axis
    """
    tr = Affine2D().scale(9, 9).rotate_deg(45)

    grid_helper = floating_axes.GridHelperCurveLinear(
        tr, extremes=(0, 1, 0, 1))

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    aux_ax = ax1.get_aux_axes(tr)

    grid_helper.grid_finder.grid_locator1._nbins = 10
    grid_helper.grid_finder.grid_locator2._nbins = 10

    return ax1, aux_ax


##########################################################
fig = plt.figure(2, figsize=(38, 14))
fig.subplots_adjust(wspace=1, left=0.5, right=2)

ax1, aux_ax1 = setup_axes1(fig, 131)

csfont = {'fontname':'BBC Reith Sans'}
hfont = {'fontname':'BBC Reith Sans'}

#add gridlines to main viz and remove ticks
ax1.grid(zorder=1,color="#484848")

plt.rc('axes',edgecolor='grey')
plt.tick_params(color='none')

#remove label colors
COLOR = '#1a1d2c'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR

#add scatter plots
aux_ax1.scatter(high_volume['Possession Regains'], high_volume['Carries per Touch'], s=400, facecolors='#f62681', edgecolors=COLOR, lw=2,zorder=4,alpha=0.7)

#add scatter plots
aux_ax1.scatter(partey['Possession Regains'], partey['Carries per Touch'], s=400, facecolors='#f62681', edgecolors='white', lw=2,zorder=5,alpha=0.7)


dx, dy = partey['Possession Regains'].values, partey['Carries per Touch'].values
for i in range(len(partey)):
    if partey.Player.iloc[i] == "Thomas Partey":
        n = aux_ax1.text(dx[i]+0.02,dy[i]+0.02,partey.Player.iloc[i],ha="center",color="white",zorder=6, **csfont, size=15, fontweight="bold")
        
n.set_path_effects([path_effects.withStroke(linewidth=4,foreground=COLOR)])

#high passer
aux_ax1.scatter(high_passer['Possession Regains'], high_passer['Carries per Touch'], s=450, facecolors='#ffff00', edgecolors='#1a1d2c', zorder=4,alpha=0.7)

#high passer
aux_ax1.scatter(high_carrier['Possession Regains'], high_carrier['Carries per Touch'], s=450, facecolors='#0892d0', edgecolors='#1a1d2c', zorder=4,alpha=0.7)

aux_ax1.scatter(longlist['Possession Regains'], longlist['Carries per Touch'], s=450, facecolors='#1ec185', edgecolors='#1a1d2c', zorder=4,alpha=0.7)


aux_ax1.plot([0.5,0.5],[0.5,1.07],color="#19c485",zorder=3, alpha=1, ls="--")
aux_ax1.plot([0.5,1.07],[0.5,0.5],color="#19c485",zorder=3, alpha=1, ls="--")

ax1.text(-7,14,"Dribble Value per 90",color="white",fontsize=25, **csfont, fontweight='bold')
ax1.text(-7,13.65,"Big 5 European Leagues | U23s | Season 2019-2020",color="white",fontsize=15, **csfont)
ax1.text(-1.35,12.85,"Double-Trouble",color="white",fontsize=20, **csfont, fontweight="bold")
ax1.text(-4.9,11.2,"Players above the line dribble\nthe ball towards the opponent's\ngoal at a frequent rate",
         color="white",fontsize=14, **csfont, horizontalalignment='left')
ax1.text(3.86,14,"Created by @jonollington",color="white",fontsize=15, **csfont)

ax1.text(6.38, 6.4, "Progressor", rotation=270, va='center', color="white", **csfont, fontsize=20, fontweight="bold")
ax1.text(-6.75, 6.4, "Carrier", rotation=90, va='center', color="white", **csfont, fontsize=20, fontweight="bold")

#add green square
rect = plt.Rectangle([0.5,0.5],0.5,0.5,color="#19c485",zorder=1,alpha=0.1)
aux_ax1.add_artist(rect)

#set figure background colour
fig.set_facecolor('#1a1d2c')
ax1.set_facecolor('#1a1d2c')

# Add fancy arrow

style="Simple,tail_width=0.4,head_width=8,head_length=10"
kw = dict(arrowstyle=style, color="white")
kwr = dict(arrowstyle=style, color="firebrick")
kwg = dict(arrowstyle=style, color="seagreen")
a = patches.FancyArrowPatch((0.5,1.25), (0.49,1.05),
                            connectionstyle="arc3,rad=0.5", **kw,zorder=20,lw=0.8) #curved arrow

b = patches.FancyArrowPatch((-0.03,0.25),(-0.03,0.1),
                            **kwr,zorder=20,lw=0.5) #straight arrow
c = patches.FancyArrowPatch((0.25,-0.035),(0.1,-0.035),
                            **kwr,zorder=20,lw=0.5) #straight arrow

d = patches.FancyArrowPatch((-0.03,0.75),(-0.03,0.9),
                            **kwg,zorder=20,lw=0.5) #straight arrow
e = patches.FancyArrowPatch((0.75,-0.035),(0.9,-0.035),
                            **kwg,zorder=20,lw=0.5) #straight arrow
aux_ax1.add_patch(a)
aux_ax1.add_patch(b)
aux_ax1.add_patch(c)
aux_ax1.add_patch(d)
aux_ax1.add_patch(e)

ax1.text(1.95,3, "- Progressive Distance per Carry +", rotation=45, va='center', color="white", **csfont, fontsize=15)
ax1.text(-4.4,3, "+ Carries per Touch -", rotation=315, va='center', color="white", **csfont, fontsize=15)


fig.text(0.501,0.12,"Only includes midfielders with > 1,000 mins",color="white",fontsize=12, **csfont, fontstyle="italic")
fig.text(0.75,0.12,"Data: FBRef | StatsBomb",color="white",fontsize=12, **csfont)
# fig.tight_layout()
plt.rcParams['savefig.facecolor']='#1a1d2c'
plt.savefig("ball_prog", dip=500, bbox_inches='tight')

In [ ]:
from PIL import Image

In [ ]:
img = Image.open('./SB_Regular_Alt.png')
img.size

In [ ]:
img = img.resize((107, 13), Image.ANTIALIAS)

In [ ]:
img